<a href="https://colab.research.google.com/github/Daarlens/ProBook/blob/main/NLP/NLP-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_4_%D0%9B%D0%B5%D0%BD%D1%87%D0%B5%D0%BD%D0%BA%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧪 **Лабораторный практикум № 4**  
# **Сравнительный анализ алгоритмов кластеризации текстовых данных на русскоязычных корпусах**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Обработка естественного языка  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Получить комплексное практическое представление о **всех основных классических методах кластеризации текстов**, основанных на статистических и статических векторных представлениях, обученных на собственном корпусе. Научиться выбирать оптимальный алгоритм в зависимости от структуры данных, вычислительных ограничений и требований к интерпретируемости.

### **Задачи:**  
1. Использовать корпус из **Лабораторной работы №1** (≥10 000 документов).  
2. Применить **методы токенизации**, включая **собственную BPE-модель**, обученную в ЛР №1.  
3. Использовать **модели векторизации**, обученные в **Лабораторной работе №2**:  
   - Word2Vec (CBOW / Skip-Gram)  
   - FastText (CBOW / Skip-Gram)  
   - GloVe  
4. Реализовать и экспериментально сравнить **все основные классические методы кластеризации**:  
   - **Центроидные**: k-Means, Mini-Batch k-Means, Spherical k-Means  
   - **Плотностные**: DBSCAN, HDBSCAN  
   - **Иерархические**: агломеративная кластеризация (single, complete, average, ward)  
   - **Вероятностные**: Gaussian Mixture Models (GMM), Latent Dirichlet Allocation (LDA)  
   - **Графовые**: спектральная кластеризация  
5. Оценить качество с помощью **внутренних и внешних метрик**.  
6. Провести **визуализацию и интерпретацию** кластеров.  
7. Исследовать **устойчивость** к шуму, дисбалансу и объёму данных.  
8. Разработать **веб-интерфейс** для анализа результатов.

---

## 📚 2. Теоретические предпосылки

Кластеризация текстов — задача **обучения без учителя**, направленная на группировку документов по семантическому сходству **без разметки**. В данной работе рассматриваются **только классические, проверенные и интерпретируемые методы**, реализованные в зрелых библиотеках (`scikit-learn`, `Gensim`, `hdbscan`).

### **2.1. Токенизация (из ЛР №1)**
- Whitespace  
- Регулярные выражения  
- **Byte Pair Encoding (BPE)** — обученная на корпусе ЛР №1 модель (через `subword-nmt` или аналог)

### **2.2. Векторизация (из ЛР №2)**
- **TF-IDF**, **BM25** — статистические веса  
- **Word2Vec**, **GloVe**, **FastText** — статические эмбеддинги, обученные в ЛР №2 на том же корпусе  
> Вектор документа = среднее по токенам. Все модели — **собственные**, не предобученные извне.

### **2.3. Классические алгоритмы кластеризации**

Все методы реализованы в следующих библиотеках:

#### **Центроидные** (`scikit-learn`):
- `KMeans` — минимизирует сумму квадратов расстояний до центроидов  
- `MiniBatchKMeans` — масштабируемая версия для больших данных  
- `SphericalKMeans` — вариант для нормализованных (косинусных) пространств

#### **Плотностные** (`scikit-learn`, `hdbscan`):
- `DBSCAN` — обнаруживает кластеры произвольной формы, устойчив к выбросам  
- `HDBSCAN` — улучшенная иерархическая версия DBSCAN, **автоматически определяет число кластеров**

#### **Иерархические** (`scikit-learn`):
- `AgglomerativeClustering` с:
  - `linkage='ward'` (требует евклидовой метрики)  
  - `linkage='average'`, `'complete'`, `'single'` (работают с `metric='cosine'`)

#### **Вероятностные**:
- `GaussianMixture` (`scikit-learn`) — мягкое назначение точек кластерам  
- `LdaModel` (`Gensim`) — генеративная тематическая модель, рассматриваемая как кластеризатор по темам

#### **Графовые** (`scikit-learn`):
- `SpectralClustering` — кластеризация на основе собственных векторов матрицы схожести

> ⚠️ Все алгоритмы работают с `numpy.ndarray`, что делает их **универсальными** для любого векторного представления.

---

## 🧪 3. Методика эксперимента

### **3.1. Подготовка данных**
- Используйте корпус из ЛР №1.  
- Реализуйте `text_preprocessing.py` с поддержкой:
  - Очистки, лемматизации (`spaCy` или `pymorphy2`),  
  - Токенизации: whitespace, regex, **BPE (из ЛР №1)**.

> 💡 **spaCy** поддерживает 64-битные Python 3.7+ на Linux, macOS, Windows. Для лемматизации русского языка может потребоваться внешний морфологический анализатор.

### **3.2. Векторизация**
- Реализуйте `text_to_vector.py` с:
  - `TfidfVectorizer` (scikit-learn)  
  - `BM25Okapi` (rank-bm25)  
  - Загрузкой моделей из ЛР №2: `Word2Vec`, `FastText`, `GloVe` (через `Gensim`)  
- Нормализуйте векторы (`L2`) при использовании cosine similarity.

> 💡 **Gensim** — сверхбыстрая библиотека для эмбеддингов (C-ядра, параллелизм, streaming). Поддерживает Python 3.8+ и работает на всех платформах.

### **3.3. Кластеризация**
Реализуйте `clustering.py`, включающий **все перечисленные выше методы**.  
Для каждого:
- Подберите гиперпараметры (`k`, `eps`, `min_samples`, `n_components` и др.)  
- Примените стратифицированную или репрезентативную выборку при оценке

### **3.4. Оценка качества**
- **Внутренние метрики**: Silhouette, Calinski-Harabasz, Davies-Bouldin  
- **Внешние метрики** (если есть тематическая разметка из ЛР №3): ARI, NMI, V-measure  
- Постройте графики зависимости метрик от параметров

### **3.5. Интерпретация**
- Для TF-IDF: топ-10 слов на кластер  
- Для эмбеддингов: ближайшие слова к центроиду (через `model.most_similar()`)  
- Визуализация: UMAP → 2D scatter plot с цветами кластеров

> 📌 **LIME и SHAP** могут применяться **только если кластеризация используется как признак в downstream-задаче** (например, кластер → признак для классификатора). В чистой кластеризации они **не применимы напрямую**, но могут использоваться для анализа косвенных эффектов.

### **3.6. AutoML для кластеризации?**
> ❗ **Важно**: **AutoML-фреймворки (Auto-sklearn, TPOT, H2O)** **не поддерживают задачи без учителя** (включая кластеризацию) в стандартной конфигурации.  
> Auto-sklearn поддерживает **только задачи с разметкой** (классификация, регрессия).  
> Следовательно, **AutoML в данной работе НЕ ИСПОЛЬЗУЕТСЯ**.

### **3.7. Веб-интерфейс**
- Используйте `Streamlit` или `Gradio` для:
  - выбора метода токенизации/векторизации/алгоритма,  
  - визуализации кластеров,  
  - просмотра содержимого кластеров.

---

## 🔍 4. Дополнительные задания

1. Сравнение влияния BPE (из ЛР №1) и whitespace на качество  
2. Использование LDA как признакового пространства для k-Means  
3. Граф кластеров: построение матрицы схожести → community detection  
4. Автоматическое определение числа кластеров (silhouette analysis, gap statistic)

---

## 📄 5. Требования к отчёту

- Укажите, что BPE и эмбеддинги — **собственные, обученные в ЛР №1 и №2**  
- Приведите **таблицу сравнения всех методов** по качеству и времени  
- Включите **визуализации** и **примеры документов** из кластеров  
- Ссылки на GitHub и веб-демо  
- Рефлексия:  
  - Какой метод кластеризации показал наилучшие результаты для русского языка?  
  - Как влияет выбор векторизации на форму кластеров?  
  - Почему AutoML не применим к кластеризации?

---

## 📖 6. Используемые библиотеки

| Библиотека | Назначение | Примечание |
|-----------|-----------|-----------|
| `Gensim` | Word2Vec, FastText, GloVe, LDA | Супербыстрая, streaming, кроссплатформенная |
| `scikit-learn` | TF-IDF, k-Means, DBSCAN, GMM, спектральная кластеризация | Стандарт де-факто |
| `hdbscan` | HDBSCAN | Для кластеров произвольной формы |
| `spaCy` | Лемматизация, базовая токенизация | Требует установки языковых моделей |
| `subword-nmt` | BPE | Для обучения и применения BPE |
| `umap-learn` | Визуализация | Альтернатива t-SNE |
| `Streamlit` / `Gradio` | Веб-интерфейс | Быстрая разработка |



## 🧪 3. Методика эксперимента

### **3.1. Подготовка данных**
- Используйте корпус из ЛР №1.  
- Реализуйте `text_preprocessing.py` с поддержкой:
  - Очистки, лемматизации (`spaCy` или `pymorphy2`),  
  - Токенизации: whitespace, regex, **BPE (из ЛР №1)**.

> 💡 **spaCy** поддерживает 64-битные Python 3.7+ на Linux, macOS, Windows. Для лемматизации русского языка может потребоваться внешний морфологический анализатор.


In [ ]:
!pip install --upgrade --quiet gensim


In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import pymorphy3
import sentencepiece as spm
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec, FastText


In [ ]:
# 1) Установка корус
!pip install corus

# 2) Скачивание корпуса Lenta
!wget -q https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

# 3) Чтение и подсчёт записей
import gzip
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = list(load_lenta(path))
print("Всего записей в корпусе Lenta:", len(records))

# 4) Посмотреть первые 5 текстов (например, заголовки и тексты)
for rec in records[:5]:
    print("URL:", rec.url)
    print("Заголовок:", rec.title)
    print("Текст:", rec.text[:200], "…")
    print("-" * 50)




In [ ]:
import gensim
from tqdm import tqdm
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def preprocess(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Токенизация (простой способ через gensim)
    tokens = gensim.utils.simple_preprocess(text, min_len=2, max_len=15)

    # 3. Лемматизация
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]

    # 4. Возврат строки
    return " ".join(lemmas)

# Применяем ко всем текстам корпуса (можно выбрать только текст, без заголовков)
texts = [rec.text for rec in records[:15000]]  # первые 15 000 документов для ЛР
preprocessed_texts = [preprocess(t) for t in tqdm(texts)]


In [ ]:
# Посмотрим первые 3 документа после предобработки
for i in range(3):
    print(preprocessed_texts[i])
    print("-" * 50)


In [ ]:
import sentencepiece as spm

# 1️⃣ Сохраняем предобработанные тексты во временный файл
with open("corpus_preprocessed.txt", "w", encoding="utf-8") as f:
    for line in preprocessed_texts:
        f.write(line + "\n")

# 2️⃣ Обучаем BPE модель
spm.SentencePieceTrainer.Train(
    input='corpus_preprocessed.txt',
    model_prefix='bpe',
    vocab_size=8000,    # размер словаря, можно увеличить
    model_type='bpe',   # тип модели
    character_coverage=1.0  # полностью покрыть все символы
)

# 3️⃣ Загружаем обученную BPE модель
sp = spm.SentencePieceProcessor()
sp.load('bpe.model')

# 4️⃣ Пример токенизации BPE
example = "Вице-премьер рассказал о смертности от рака в регионах"
print(sp.encode(example, out_type=str))


In [ ]:
from gensim.models import Word2Vec, FastText

# Список токенов для каждого документа
sentences = [doc.split() for doc in preprocessed_texts]

# 1️⃣ Word2Vec
w2v = Word2Vec(sentences, vector_size=100, window=5, sg=1, min_count=2, workers=4)
print("Пример похожих слов к 'рак':", w2v.wv.most_similar("рак", topn=5))

# 2️⃣ FastText
ft = FastText(sentences, vector_size=100, window=5, sg=1, min_count=2, workers=4)
print("Пример похожих слов к 'рак':", ft.wv.most_similar("рак", topn=5))

# 3️⃣ Вектор документа = среднее по токенам
import numpy as np

def doc_vector(model, doc):
    vecs = [model.wv[word] for word in doc.split() if word in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

X_w2v = np.array([doc_vector(w2v, doc) for doc in preprocessed_texts])
X_ft = np.array([doc_vector(ft, doc) for doc in preprocessed_texts])

print("Word2Vec матрица:", X_w2v.shape)
print("FastText матрица:", X_ft.shape)


In [ ]:
%%writefile text_preprocessing.py
import gensim
from pymorphy3 import MorphAnalyzer
import sentencepiece as spm

morph = MorphAnalyzer()
sp = spm.SentencePieceProcessor()
sp.load("bpe.model")  # если есть готовая BPE

def preprocess(text):
    tokens = gensim.utils.simple_preprocess(text)
    lemmas = [morph.parse(t)[0].normal_form for t in tokens]
    return " ".join(lemmas)

def tokenize_bpe(text):
    return sp.encode(text, out_type=str)



In [ ]:
from text_preprocessing import preprocess, tokenize_bpe

doc = "Вице-премьер рассказал о смертности"
tokens = preprocess(doc)
bpe_tokens = tokenize_bpe(doc)

print("Preprocessed:", tokens)
print("BPE tokens:", bpe_tokens)


In [ ]:
from text_preprocessing import preprocess, tokenize_bpe
from tqdm import tqdm

# Берём текстовые части документов
texts = [rec.text for rec in records[:15000]]

# 1️⃣ Предобработка (очистка + лемматизация)
preprocessed_texts = [preprocess(doc) for doc in tqdm(texts)]

# 2️⃣ BPE-токенизация всего корпуса
bpe_texts = [tokenize_bpe(doc) for doc in tqdm(preprocessed_texts)]

# 3️⃣ Проверка первых документов
for i in range(3):
    print("Документ:", preprocessed_texts[i])
    print("BPE токены:", bpe_texts[i][:20])
    print("-" * 50)




### **3.2. Векторизация**
- Реализуйте `text_to_vector.py` с:
  - `TfidfVectorizer` (scikit-learn)  
  - `BM25Okapi` (rank-bm25)  
  - Загрузкой моделей из ЛР №2: `Word2Vec`, `FastText`, `GloVe` (через `Gensim`)  
- Нормализуйте векторы (`L2`) при использовании cosine similarity.

> 💡 **Gensim** — сверхбыстрая библиотека для эмбеддингов (C-ядра, параллелизм, streaming). Поддерживает Python 3.8+ и работает на всех платформах.

In [ ]:
%%writefile text_to_vector.py
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from gensim.models import Word2Vec, FastText
from sklearn.preprocessing import normalize

# --------------------------
# TF-IDF
# --------------------------
def tfidf_vectorize(docs, max_features=10000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(docs)
    X = normalize(X, norm='l2')  # L2-нормализация
    return X, vectorizer

# --------------------------
# BM25
# --------------------------
def bm25_vectorize(tokenized_docs):
    bm25 = BM25Okapi(tokenized_docs)
    return bm25

# --------------------------
# Word2Vec / FastText
# --------------------------
def train_word2vec(sentences, vector_size=100, window=5, sg=1, min_count=2):
    model = Word2Vec(sentences, vector_size=vector_size, window=window,
                     sg=sg, min_count=min_count, workers=4)
    return model

def train_fasttext(sentences, vector_size=100, window=5, sg=1, min_count=2):
    model = FastText(sentences, vector_size=vector_size, window=window,
                     sg=sg, min_count=min_count, workers=4)
    return model

# Среднее по токенам для документа
def doc_vector(model, doc_tokens):
    vecs = [model.wv[token] for token in doc_tokens if token in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)


In [ ]:
from text_to_vector import tfidf_vectorize, bm25_vectorize, train_word2vec, train_fasttext, doc_vector
from tqdm import tqdm

# 1️⃣ TF-IDF (работает на предобработанных текстах)
X_tfidf, tfidf_vectorizer = tfidf_vectorize(preprocessed_texts, max_features=10000)
print("TF-IDF матрица:", X_tfidf.shape)

# 2️⃣ BM25 (требует токенов)
tokenized_docs = [doc.split() for doc in preprocessed_texts]
bm25 = bm25_vectorize(tokenized_docs)
print("BM25 готов")

# 3️⃣ Word2Vec
w2v_model = train_word2vec(tokenized_docs, vector_size=100)
X_w2v = np.array([doc_vector(w2v_model, doc) for doc in tqdm(tokenized_docs)])
print("Word2Vec матрица:", X_w2v.shape)

# 4️⃣ FastText
ft_model = train_fasttext(tokenized_docs, vector_size=100)
X_ft = np.array([doc_vector(ft_model, doc) for doc in tqdm(tokenized_docs)])
print("FastText матрица:", X_ft.shape)


In [ ]:
%%writefile text_to_vector.py
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from gensim.models import Word2Vec, FastText
from sklearn.preprocessing import normalize
import gensim.downloader as api

# --------------------------
# TF-IDF
# --------------------------
def tfidf_vectorize(docs, max_features=10000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(docs)
    X = normalize(X, norm='l2')  # L2-нормализация
    return X, vectorizer

# --------------------------
# BM25
# --------------------------
def bm25_vectorize(tokenized_docs):
    bm25 = BM25Okapi(tokenized_docs)
    return bm25

# --------------------------
# Word2Vec / FastText
# --------------------------
def train_word2vec(sentences, vector_size=100, window=5, sg=1, min_count=2):
    model = Word2Vec(sentences, vector_size=vector_size, window=window,
                     sg=sg, min_count=min_count, workers=4)
    return model

def train_fasttext(sentences, vector_size=100, window=5, sg=1, min_count=2):
    model = FastText(sentences, vector_size=vector_size, window=window,
                     sg=sg, min_count=min_count, workers=4)
    return model

def doc_vector(model, doc_tokens):
    vecs = [model.wv[token] for token in doc_tokens if token in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

# --------------------------
# GloVe
# --------------------------
def glove_vectorize(tokenized_docs, model_name="glove-wiki-gigaword-100"):
    glove_model = api.load(model_name)
    vectors = []
    for doc in tokenized_docs:
        vecs = [glove_model[word] for word in doc if word in glove_model]
        vectors.append(np.mean(vecs, axis=0) if vecs else np.zeros(glove_model.vector_size))
    vectors = normalize(np.array(vectors), norm='l2')
    return vectors, glove_model


In [ ]:
from text_to_vector import tfidf_vectorize, bm25_vectorize, train_word2vec, train_fasttext, doc_vector, glove_vectorize
from tqdm import tqdm

# 1️⃣ TF-IDF
X_tfidf, tfidf_vectorizer = tfidf_vectorize(preprocessed_texts)

# 2️⃣ BM25
tokenized_docs = [doc.split() for doc in preprocessed_texts]
bm25 = bm25_vectorize(tokenized_docs)

# 3️⃣ Word2Vec
w2v_model = train_word2vec(tokenized_docs)
X_w2v = np.array([doc_vector(w2v_model, doc) for doc in tqdm(tokenized_docs)])

# 4️⃣ FastText
ft_model = train_fasttext(tokenized_docs)
X_ft = np.array([doc_vector(ft_model, doc) for doc in tqdm(tokenized_docs)])

# 5️⃣ GloVe
X_glove, glove_model = glove_vectorize(tokenized_docs)
print("Все матрицы созданы!")
